# Data Access Groups (DAGs) {#sec-DAG}

::: {.content-hidden when-format="html"}

## Project Setup

Install and load the necessary packages

In [1]:
#| echo: false
#| output: false
import os
os.environ['R_HOME'] = f'C:/Users/{os.environ.get('USERNAME')}/Miniconda3/envs/r_python_jl/Lib/R'

In [2]:
#| echo: false
#| output: false
%load_ext rpy2.ipython
# only have to run once to allow the R magic command

C:\Users\CMD3303\Miniconda3\envs\r_python_jl\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


::: {.panel-tabset}

#### R

In [3]:
%%capture 
%%R

library("dplyr")
library("jsonlite")
library("tidyr")
library("REDCapR")
library("knitr")
library("remotes")
library("gt")

In [4]:
%%capture --no-display --no-stdout
%%R

version <- packageVersion("REDCapR")
version

[1] '1.1.9005'


In this project, we will use the bleeding edge version of REDCapR available on Github

In [5]:
%%capture --no-display --no-stdout
%%R

# Detach REDCapR if already loaded, and download the latest version
if (version!='1.1.9005') {
    detach("package:REDCapR", unload=TRUE)
    remotes::install_github("OuhscBbmc/REDCapR")
    library("REDCapR")
    print(packageVersion("REDCapR"))
}

else {
    print("REDCapR package up to date")
}

[1] "REDCapR package up to date"


#### Python

In [6]:
import redcap
import json
import pandas as pd

:::

Assign your project URL and Token

::: {.panel-tabset}

#### R

In [7]:
%%R
path = paste0("C:/Users/", Sys.getenv("USERNAME"), '/json_api_data.json')
token <- jsonlite::fromJSON(path)$dev_token$'308'
url <- "https://dev-redcap.doh.wa.gov/api/"

#### Python

In [8]:
path_to_json = f"C:/Users/{os.environ.get('USERNAME')}/json_api_data.json"
api_key = json.load(open(path_to_json))
api_token = api_key['dev_token']['308']
api_url = api_key['dev_url']
project = redcap.Project(api_url, api_token)

:::

:::

In addition to user roles for access control, DAGs control which records each user can access. Users assigned to a DAG can only access records assigned to that particular DAG, and are blinded to records outside of their group. DAGs are particularly useful in multi-site or multi-jurisdictional projects to restrict sites from viewing records from other sites.  

Users can be assigned to a DAG after being added to the project. Users can be in multiple DAGs. Users not assigned to any DAG have global access and can see all records in the project. 

Records are assigned to DAG's using the 'Record Status Dashboard' in REDCap. In this project, one DAG group was created called 'records1_2' that contains Record IDs 1 & 2. 

::: {.panel-tabset}

#### R

`redcap_dag_read()`

**View the DAGs and their unique `data_access_group_id`:**

In [9]:
%%capture
%%R 
dag <- redcap_dag_read(redcap_uri = url, token)$data

::: {.content-hidden when-format="html"}

In [10]:
%%R
dag_tbl <- gt(head(dag))
gt::gtsave(dag_tbl, filename = 'export_dag1.html', path = "./files/export_files/")

:::

<p align="center"><iframe width="100%" height="150" src="./files/export_files/export_dag1.html" title="Quarto Documentation"></iframe></p>

**Use `redcap_read_oneshot` to view record DAG assignments:**

In [11]:
%%capture
%%R
data_dag <- redcap_read_oneshot(
    redcap_uri = url, 
    token = token,
    export_data_access_groups = TRUE, 
    fields = "record_id")$data

::: {.content-hidden when-format="html"}

In [12]:
%%R
data_dag_tbl <- gt(head(data_dag))
gt::gtsave(data_dag_tbl, filename = 'export_dag2.html', path = "./files/export_files/")

:::

<p align="center"><iframe width="100%" height="340" src="./files/export_files/export_dag2.html" title="Quarto Documentation"></iframe></p>

For the REDCapR package, refer to the `data_user` table printed in @sec-users to see user DAG assignments. 

#### Python

`export_dags()`, `export_user_dag_assignment()`

**View the DAGs and their unique `data_access_group_id`:**

In [13]:
project.export_dags(format_type='df') 

,data_access_group_name,unique_group_name,data_access_group_id
0,Limited Access,limited_access,2707
1,records1_2,records1_2,2716


**View users' DAG assignments:**

In [14]:
project.export_user_dag_assignment(format_type='df') #exports users and their assigned DAGs (username and DAG)

,username,redcap_data_access_group
0,alexey.gilman@doh.wa.gov,records1_2
1,caitlin.drover@doh.wa.gov,NaN
2,emily.pearman@doh.wa.gov,NaN


**Use `export_records` to view which records belong to which data access group.**

In [15]:
dag = project.export_records(format_type = 'df', export_data_access_groups = True, fields = 'record_id')
dag.head()

redcap_repeat_instrument  \
record_id redcap_event_name                               
1         personal_info_arm_1                       NaN   
          notifications_arm_1                       NaN   
          case_intake_arm_1                         NaN   
2         personal_info_arm_1                       NaN   
          notifications_arm_1                       NaN   

                               redcap_repeat_instance redcap_data_access_group  
record_id redcap_event_name                                                     
1         personal_info_arm_1                     NaN               records1_2  
          notifications_arm_1                     NaN               records1_2  
          case_intake_arm_1                       1.0               records1_2  
2         personal_info_arm_1                     NaN               records1_2  
          notifications_arm_1                     NaN               records1_2

**Note:** Even though only `record_id` was specified for export under the `fields` argument, PyCap will automatically include all the variables that make the unique_key (`record_id`, `redcap_event_name`, `redcap_repeat_instrument`, `redcap_repeat_instance`).

:::